## LDA(Latent Dirichlet Allocation): scikit-learn

### 데이터 준비

In [ ]:
import pandas as pd

In [ ]:
news = pd.read_csv('./뉴스.csv')
#3월 2일자 기사 두개 들어가있음 -> start_date = 0301 end_date = 0301 로 하면 됨 

In [132]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16122 entries, 0 to 16548
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   news_id        16122 non-null  int64  
 1   title          16122 non-null  object 
 2   writer         13701 non-null  object 
 3   content        16122 non-null  object 
 4   writed_at      16122 non-null  object 
 5   url            0 non-null      float64
 6   main_category  16122 non-null  object 
 7   sub_category   16122 non-null  object 
 8   platform       16122 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.2+ MB


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
news.dropna(subset=['content'], inplace=True) #content null 값 행 삭제 

In [131]:
# 중복된 기사 날리기
news.drop_duplicates(subset=['content'], inplace=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 전처리 필요 부분 확인

In [99]:
news['content'].iloc[0]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 전담할 '우주항공청'을 출범시킬 계획이다. 외부 전문가 채용 비중 제한을 없애고, 우수 인력을 유치하고자 주식백지신탁 예외까지 허용하는 등 우주항공청의 전문성을 높이기 위한 다양한 특례까지 마련된다. 과학기술정보통신부는 우주항공청 설립의 근거가 되는 '우주항공청의 설치 및 운영에 관한 특별법' 제정안을 2일부터 입법예고한다고 밝혔다. 특별법은 우주항공분야의 정책과 R&D, 산업육성 등을 총괄하는 중앙행정기관으로 우주항공청을 설치하고, 전문적이고 유연한 조직으로 운영하기 위한 원칙과 기능, 특례 등까지 명시하고 있다. 먼저 우주항공 정책 거버넌스를 구축하기 위해 과기정통부, 산업통상자원부 등의 부처가 개별적으로 수행하던 우주항공 관련 기술개발, 산업육성 지원, 인재양성 및 우주위험 대비 등의 기능을 모두 우주항공청으로 일원화했다. 이를 위해 기존 과기정통부 장관, 산지부 장관 등이 소관하던 우주항공 관련 법률인 '우주개발진흥법', '항공우주산업촉진법', '천문법' 등을 우주항공청장이 담당하도록 부칙을 통해 개정할 방침이다. 국가우주위원회 위원장은 국무총리에서 대통령으로 격상해 위상과 기능을 강화한다. 우주항공청장도 국가우주위에 새롭게 위원으로 추가해 실무위원회 위원장을 맡게 하는 등 우주항공청이 우주경제 시대를 여는 컨트롤 타워 역할을 하도록 했다. 우주항공청 조직의 경우 과기정통부 장관 소속으로 하되 전문성을 가진 조직으로 구성된다. 특히 우주항공청 조직 구성 원칙의 핵심은 유연하고 자율적인 운영이다. 이를 위해 우주항공청 내에 설치되는 별도의 본부가 우주항공 R&D와 산업 활성화를 전담하게 된다."

In [ ]:
# ▲ 없애기 
text = re.sub(r'[▲]','', text)

#뉴스기사 출처 ( = ), [ = ] 형태 없애기 코드 '\(.*=.*\) ' 
text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))

#[] 안에 들어가는 거 다 없애기? -> 그러면 안됨.. 

# 괄호 안 기자로 끝나는 거 지우기 
#re.sub(r'\[.*기자\]', '', text)

text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

 # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)

In [96]:
a = news['content'].str.contains('페이스북 /LeYN1 트위터 @yonhap_graphics')

print((news['content'].loc[a]))

7528     \n ,(서울=연합뉴스) 원형민 기자 = 정부가 미래산업의 성장동력을 확보하기 위한...
9133     \n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실...
9937     \n ,(서울=연합뉴스) 원형민 기자 = 서방이 우크라이나 침공에 대한 제재로 러시...
16843    \n ,(서울=연합뉴스) 반종빈 기자 = 일본은 16일 한국에 대한 반도체 핵심 소...
Name: content, dtype: object


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
news['content'].iloc[9133]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\n ,(서울=연합뉴스) 원형민 기자 = 한국은행이 22일 발표한 '2월 기업경기실사지수(BSI)' 조사 결과에 따르면 이달 제조업 업황 BSI는 전월보다 3포인트 하락한 63을 기록했다., , 페이스북 /LeYN1 트위터 @yonhap_graphics,\n "

### tokenization

In [ ]:
!pip3 install mecab-python3

In [ ]:
!pip install konlpy

In [134]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)

    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    stopwords = ['하는', '있다', '있는', '위해', '통해', '한다', '때문'] # 불용어 사전  
    for word, pos in Okt_morphs:
        if pos in ['Adjective', 'Verb', 'Noun', 'Alpha', 'Number'] and word not in stopwords:  # 이 경우에는 형용사, 동사, 명사, 영어, 숫자만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# news['content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


16122
정부 올해 중 우리나라 우주 항공 분야 연구개발 R D 전담 할 우주 항공 청 출범 시킬 계획 외부 전문가 채용 비중 제한 없애고 우수 인력 유치하고자 주식 신탁 예외 허용 등 우주 항공 청 전문성 높이기 위 다양한 특례 마련 된다 과학기술 정보통신부 우주 항공 청 설립 근거 되는 우주 항공 청 설치 및 운영 관 특별법 정안 2일 입법 예고 한다고 밝혔다 특별법 우주 항공 분야 정책 R D 산업 육성 등 총괄 중앙 행정기관 우주 항공 청 설치 전문 유연한 조직 운영 하기 위 원칙 기능 특례 등 명시 먼저 우주 항공 정책 거버넌스 구축 하기 과 정통부 산업 통상 자원부 등 부처 개별 수행 하던 우주 항공 관련 기술 개발 산업 육성 지원 인재 양성 및 우주 위험 대비 등 기능 모두 우주 항공 청 일원 화했다 이를 기존 과 정통부 장관 산지 부 장관 등 소관 하던 우주 항공 관련 법률 우주개발 진흥 법 항공우주 산업 촉진법 문법 등 우주 항공 청장 담당 하도록 부칙 개정 할 방침 국가 우주 위원회 위원장 국무총리 대통령 격상 해 위상 기능 강화한다 우주 항공 청장 국가 우주 위 새롭게 위원 추가 해 실무 위원회 위원장 맡게 등 우주 항공 청 우주 경제 시대 여 컨트롤 타워 역할 하도록 했다 우주 항공 청 조직 경우 과 정통부 장관 소속 되 전문성 가진 조직 구성 된다 우주 항공 청 조직 구성 원칙 핵심 유연하고 자율 운영 이를 우주 항공 청 내 설치 되는 별도 본부 우주 항공 R D 와 산업 활성화 전담 하게 된다


In [136]:
#단어가 1-20개만 포함된 corpus 삭제 
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus.split())) < 21:   # 같은 단어 1-20개만 반복되는 corpus도 지우기 위해 set()을 사용
        news.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)
    
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

print(len(tokenized_list))
print(len(news))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


15901
15901


### vectorization & LDA

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#### vectorization

In [137]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.8, max_features=5000, min_df=20, ngram_range=(1,2))
    # 20개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외, 단어 수(백터 수)는 5000
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape) #문서의 수, 단어의 수

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CountVectorizer Shape: (15901, 5000)


#### topic modeling: LDA

In [140]:
lda = LatentDirichletAllocation(n_components=10)  # 토픽 수는 10개로 설정
lda.fit(feat_vect)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation()

#### 토픽별 연관어 출력

In [141]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names_out()를 통해 추출
feature_names = count_vectorizer.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
매출 지난해 대비 시장 증가 사업 기록 영업 이익 성장
Topic # 1
기술 기업 데이터 사업 지원 디지털 서비스 개발 클라우드 솔루션
Topic # 2
전자 갤럭시 제품 23 삼성 고객 lg 스마트폰 출시 시리즈
Topic # 3
lg 로봇 서비스 기술 유플러스 lg 유플러스 통신 sk 버스 고객
Topic # 4
서비스 자산 금융 정보 카카오 개인정보 거래 가상 대한 체인
Topic # 5
게임 콘텐츠 진행 다양한 대표 밝혔다 예정 이번 개발 제공
Topic # 6
반도체 투자 기업 산업 미국 생산 기술 sk 중국 한국
Topic # 7
kt 대표 카카오 후보 경영 sm 대표이사 사장 주주 이사회
Topic # 8
연구 치료 개발 환자 의료 바이오 교수 결과 병원 됐다
Topic # 9
ai gpt 인공 지능 인공 지능 서비스 구글 기술 검색 모델


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### LDA 시각화

In [104]:
!pip install pyLDAvis # pyLDAvis 라이브러리를 이용한 시각화 

In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer.get_feature_names_out())
pyLDAvis.display(vis) 

In [ ]:
pyLDAvis.show(vis) 

### 문서별 토픽 할당

#### 1. 각 문서별로 가장 가까운 topic으로 할당

In [142]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌 
doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num,Topic,Percentage
0,0,6,0.835958
1,1,6,0.761948
2,2,6,0.797824
3,3,9,0.975848
4,4,6,0.802876


In [143]:
doc_topic_df = doc_topic_df.join(news)
doc_topic_df.head() #실제 뉴스기사와 join

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,6,0.835958,346630,"정부, '우주항공청' 연내 개청…백지신탁 예외 등 특혜 쏟는다",NaN,\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 ...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
1,1,6,0.761948,346633,우주항공청 전문가 영입 `연봉 10억·외국인 허용`,이준기,\n '한국형 NASA(미 항공우주국)'로 출범하는 우주항공청이 최고의 민간 전문가...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
2,2,6,0.797824,347804,"박정호의 투자유치 '매직'…""SK쉴더스 IPO보다 더 이득""",선한결,"\n ""SK쉴더스의 기업공개(IPO)를 철회해서 아쉬웠는데, IPO보다 더 높은 가...",2023-03-01 18:03:05,NaN,IT/과학,IT 일반,네이버
3,3,9,0.975848,347806,글·그림·로고 제작·번역...인공지능에 다 맡겨 보니,NaN,\n 최근 텍스트 생성 인공지능(AI) 챗봇 챗GPT(ChatGPT)를 이용해 만든...,2023-03-01 18:02:03,NaN,IT/과학,IT 일반,네이버
4,4,6,0.802876,347807,"LG엔솔-혼다, 美 배터리 공장 '첫삽'…기공식 개최",이한얼,\n LG에너지솔루션은 28일(현지시간) 오하이오 주 파이에트 카운티 제퍼슨빌 인근...,2023-03-01 18:26:01,NaN,IT/과학,IT 일반,네이버


#### 2. 토픽별 문서 수 계산

In [144]:
doc_topic_df.groupby('Topic')[['Doc_Num']].count()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num
Topic,
0,1244
1,3172
2,1365
3,1158
4,1654
5,1953
6,1556
7,800
8,1553


#### 3. 토픽별로, 가장 높은 확률로 할당된 문서 top1 확인(이걸로 메인 뉴스 뽑기)

In [145]:
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    print(top_pr_topics['content'].iloc[0])
    print(top_pr_topics['content'].iloc[1])
    print(top_pr_topics['content'].iloc[2], '\n')

Topic # 0 -----------------------------

 쿠팡이 지난해 4분기 최대 분기 매출, 지난 3분기에 이은 연속 흑자를 달성하며 '계획된 적자' 전략을 입증했다. 국내 이커머스 사업자 지위를 공고히한 쿠팡은 향후 온·오프라인 통합 유통 시장에서도 선두를 확보하겠다는 포부다. 쿠팡은 지난해 4분기 매출 7조2천404억원(53억2천677만 달러), 영업이익 1천133억원(8천340만 달러)을 내며, 2분기 연속 흑자를 달성했다. 글로벌 경기 침체로 인한 소비 심리 위축에도 로켓배송, 로켓프레시 등 서비스 강화로 충성 고객을 확보해 성장세로 이어졌다는 분석이다. 지난해 4분기 와우 멤버십 회원 수는 1천100만명을 기록, 전년 대비 약 200만명 증가했다. 지난해 연매출은 전년 대비 26% 증가한 26조5천917억원(205억8천261만달러·연환율 ), 영업적자는 1천447억원(1억1천201만달러)을 기록했다. 조정 에비타(EBITDA, 상각전 영업이익) 기준으로는 4천925억원(3억8천121만달러)을 기록하며, 올해 연간 흑자 달성 가능성에도 청신호가 켜졌다는 전망이 나온다. 쿠팡은 28일(현지시간) 지난해 4분기 매출 7조2천404억원(53억2천677만달러)을 올렸다고 미국 증권거래위원회(SEC)에 공시했다. 전년 대비 21% 늘어난 수준이다. 분기 매출로는 최초로 7조원을 돌파했다. 4분기 영업이익은 전 분기 대비 9% 증가한 1천133억원(8천340만달러)으로, 지난 3분기 영업익(1천37억원)에 이어 2분기 연속 1천억원대 영업흑자를 거뒀다. 당기순이익은 1천387억원(1억206만달러)로 전 분기 대비 14% 증가했다. 사상 최대 분기 매출과 2분기 연속 영업흑자에 힘입어, 쿠팡은 지난해 26조5천917억원(205억8천261만 달러·연환율 ) 매출을 올렸다. 전년 대비 26% 늘어난 역대 최대 매출을 기록한 것. 지난해 영업적자 규모는 1천447억원(1억1천201만달러)으로, 전년의 1조7천97억원(14억9천396만달러)과 비교해 

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# LDA: gensim

## tokenization

In [111]:
import konlpy
import re

def tokenize_korean_text(text):
    text = re.sub(r'[▲]','', text) 
    text = re.sub(r'\(.*=.*\) ', '', re.sub(r'\[.*=.*\] ', '', text))
    text = re.sub(r'페이스북 /LeYN1 트위터 @yonhap_graphic', '', text)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)
    
    words = []
    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
            words.append(word)
    ## word를 이어붙인 string 형태가 아닌 word의 list를 return해주는 게 sklearn lda 준비 과정과의 차이
            
    return words


tokenized_list = []

for text in news['content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


16700
['정부', '올해', '중', '우리나라', '우주', '항공', '분야', '연구개발', '전담', '할', '우주', '항공', '청', '출범', '시킬', '계획', '외부', '전문가', '채용', '비중', '제한', '없애고', '우수', '인력', '유치하고자', '주식', '신탁', '예외', '허용', '하는', '등', '우주', '항공', '청', '전문성', '높이기', '위', '다양한', '특례', '마련', '된다', '과학기술', '정보통신부', '우주', '항공', '청', '설립', '근거', '되는', '우주', '항공', '청', '설치', '및', '운영', '관', '특별법', '정안', '입법', '예고', '한다고', '밝혔다', '특별법', '우주', '항공', '분야', '정책', '산업', '육성', '등', '총괄', '하는', '중앙', '행정기관', '우주', '항공', '청', '설치', '전문', '유연한', '조직', '운영', '하기', '위', '원칙', '기능', '특례', '등', '명시', '있다', '먼저', '우주', '항공', '정책', '거버넌스', '구축', '하기', '위해', '과', '정통부', '산업', '통상', '자원부', '등', '부처', '개별', '수행', '하던', '우주', '항공', '관련', '기술', '개발', '산업', '육성', '지원', '인재', '양성', '및', '우주', '위험', '대비', '등', '기능', '모두', '우주', '항공', '청', '일원', '화했다', '이를', '위해', '기존', '과', '정통부', '장관', '산지', '부', '장관', '등', '소관', '하던', '우주', '항공', '관련', '법률', '우주개발', '진흥', '법', '항공우주', '산업', '촉진법', '문법', '등', '우주', '항공', '청장', '담당', '하도록', '부칙', '통해', '개정', '할', '방침'

In [112]:
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus)) < 3:   # corpus 자체가 list 형태인게 sklearn lda 준비 과정과의 차이
        news.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)
    
for corpus in drop_corpus:
    tokenized_list.remove(corpus)

news.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## vectorization & LDA

In [50]:
# bigram 생성에 필요한 library
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# vectorize & lda에 필요한 library
from gensim import corpora
from gensim.models.ldamodel import LdaModel

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to

In [51]:
# Build the bigram models
bigram = Phrases(tokenized_list, min_count=4, threshold=10)
bigram_mod = Phraser(bigram)
 
# See example
print(bigram_mod[tokenized_list[0]])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['서울_뉴시스', '정부', '올해', '중', '우리나라', '우주_항공', '분야', '연구개발', '전담', '할', '우주_항공', '청', '출범', '시킬_계획', '외부_전문가', '채용', '비중', '제한', '없애고', '우수_인력', '유치하고자', '주식_신탁', '예외_허용', '하는', '등', '우주_항공', '청', '전문성_높이기', '위', '다양한', '특례', '마련', '된다', '과학기술_정보통신부', '우주_항공', '청_설립', '근거', '되는', '우주_항공', '청_설치', '및', '운영', '관_특별법', '정안_입법', '예고', '한다고_밝혔다', '특별법_우주', '항공', '분야', '정책', '산업_육성', '등', '총괄', '하는', '중앙_행정기관', '우주_항공', '청_설치', '전문_유연한', '조직', '운영', '하기_위', '원칙', '기능', '특례', '등', '명시', '있다', '먼저', '우주_항공', '정책', '거버넌스', '구축', '하기_위해', '과_정통부', '산업_통상', '자원부', '등', '부처_개별', '수행_하던', '우주_항공', '관련', '기술', '개발', '산업_육성', '지원', '인재_양성', '및', '우주_위험', '대비', '등', '기능', '모두', '우주_항공', '청', '일원_화했다', '이를_위해', '기존', '과_정통부', '장관', '산지', '부_장관', '등', '소관', '하던', '우주_항공', '관련', '법률_우주개발', '진흥_법', '항공우주_산업', '촉진법_문법', '등', '우주_항공', '청장', '담당', '하도록', '부칙', '통해', '개정', '할_방침', '국가', '우주', '위원회_위원장', '국무총리_대통령', '격상', '해', '위상', '기능', '강화한다', '우주_항공', '청장', '국가', '우주', '위', '새롭게', '위원', '추가', '해', '실무', '위원회

In [52]:
# tokenized_list의 모든 문서에 대해 bigram을 생성해줌
words_bigram = [bigram_mod[doc] for doc in tokenized_list]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
dictionary = corpora.Dictionary(words_bigram)  # 각 단어에 번호를 할당해줌
    # bigram 포함하는 과정을 생략하고 싶으면, 그냥 바로 여기에 tokenized_list를 넣어주면 됨

dictionary.filter_extremes(no_below=10, no_above=0.80)   
    # 10개의 문서 미만으로 등장하는 단어는 제외, 전체의 80% 이상으로 자주 등장하는 단어는 제외

corpus = [dictionary.doc2bow(text) for text in words_bigram]
print(corpus[0]) # 첫번째 corpus를 테스트로 출력: 몇 번째 단어가 몇 번 나왔는지 저장되어 있음

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 17), (42, 2), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 6), (67, 1), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1)]


In [54]:
ldamodel = LdaModel(corpus, num_topics=6, id2word=dictionary, passes=20, iterations=500)  # 토픽 수: 6

ldamodel.print_topics(num_words=7)  # num_words=10이 default

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.003*"환자" + 0.003*"입니다" + 0.002*"병원" + 0.002*"의료" + 0.002*"있습니다" + 0.002*"치료" + 0.002*"애플"'),
 (1,
  '0.006*"배터리" + 0.004*"과_정통부" + 0.003*"에너지" + 0.003*"조성" + 0.003*"사업자" + 0.002*"시설" + 0.002*"텔레콤"'),
 (2,
  '0.004*"인수" + 0.004*"하이브" + 0.004*"사장" + 0.003*"이사회" + 0.003*"주주" + 0.003*"임_직원" + 0.003*"영업_이익"'),
 (3,
  '0.006*"로봇" + 0.005*"구글" + 0.004*"네이버" + 0.004*"메타_버스" + 0.004*"학습" + 0.004*"챗봇" + 0.003*"검색"'),
 (4,
  '0.006*"가상_자산" + 0.005*"블록_체인" + 0.003*"일본" + 0.003*"애플" + 0.003*"은행" + 0.003*"거래소" + 0.003*"결제"'),
 (5,
  '0.005*"유플러스" + 0.003*"기기" + 0.003*"갤럭시" + 0.003*"시리즈" + 0.002*"디스플레이" + 0.002*"디자인" + 0.002*"세대"')]

## LDA 시각화

In [55]:
import pyLDAvis.gensim  # gensim의 ldamodel에 최적화된 라이브러리

vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


## 문서별 토픽 할당

In [57]:
doc_per_topic_list = []

for n in range(len(corpus)):
    doc_topic = ldamodel[corpus[n]]
    doc_topic = sorted(doc_topic, key=lambda x: (x[1]), reverse=True)
    topic_most_pr = doc_topic[0][0]
    topic_pr = doc_topic[0][1]
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])
    
doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

# 실제 review 내용과 join
doc_topic_df = doc_topic_df.join(news)  

doc_topic_df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Doc_Num,Topic,Percentage,news_id,title,writer,content,writed_at,url,main_category,sub_category,platform
0,0,2,0.675528,346630,"정부, '우주항공청' 연내 개청…백지신탁 예외 등 특혜 쏟는다",NaN,\n [서울=뉴시스] 정부가 올해 중 우리나라의 우주항공 분야 연구개발(R&D)을 ...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
1,1,2,0.726458,346633,우주항공청 전문가 영입 `연봉 10억·외국인 허용`,이준기,\n '한국형 NASA(미 항공우주국)'로 출범하는 우주항공청이 최고의 민간 전문가...,2023-03-02 00:00:00,NaN,IT/과학,IT 일반,네이버
2,2,2,0.764771,347804,"박정호의 투자유치 '매직'…""SK쉴더스 IPO보다 더 이득""",선한결,"\n ""SK쉴더스의 기업공개(IPO)를 철회해서 아쉬웠는데, IPO보다 더 높은 가...",2023-03-01 18:03:05,NaN,IT/과학,IT 일반,네이버
3,3,5,0.437542,347806,글·그림·로고 제작·번역...인공지능에 다 맡겨 보니,NaN,\n 최근 텍스트 생성 인공지능(AI) 챗봇 챗GPT(ChatGPT)를 이용해 만든...,2023-03-01 18:02:03,NaN,IT/과학,IT 일반,네이버
4,4,1,0.365018,347807,"LG엔솔-혼다, 美 배터리 공장 '첫삽'…기공식 개최",이한얼,\n LG에너지솔루션은 28일(현지시간) 오하이오 주 파이에트 카운티 제퍼슨빌 인근...,2023-03-01 18:26:01,NaN,IT/과학,IT 일반,네이버
